In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

### Data

In [3]:
n=1000
p=5

x=torch.randn(n, p).to(device)
w_true=torch.tensor([i/0.5 for i in range(p)], dtype=torch.float, device=device)

noise_mean=0
noise_std=0.2

y=x@w_true+ noise_std*torch.randn(n, dtype=torch.float, device=device) + noise_mean
y=y.reshape(-1, 1).to(device)

x.shape, y.shape

(torch.Size([1000, 5]), torch.Size([1000, 1]))

### Without scheduler

In [32]:
def train_model(scheduler=False):
    
    torch.manual_seed(42)
    lr=0.25
    model = nn.Sequential(nn.Linear(p, 1)).to(device)
    loss_fn=nn.MSELoss(reduction='mean')
    optimizer = optim.SGD(model.parameters(), lr=lr)
    if scheduler=='StepLR':
        schdler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=.9)
    if scheduler=='CosineAnnealingLR':
        schdler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
    if scheduler=='ExponentialLR':
        schdler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
    if scheduler=='ReduceLROnPlateau':
        schdler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=10, factor=0.1)
    
    n_epoch=100

    for epoch in range(n_epoch):
        model.train()
        loss=loss_fn(model(x), y)
        optimizer.zero_grad(set_to_none=True) 
        loss.backward()
        optimizer.step() 
        if scheduler=='ReduceLROnPlateau':
            schdler.step(loss)
        elif scheduler:
            schdler.step()
        else:
            continue
    print(f"\n\nEpoch :: {epoch+1} ")
    if scheduler:
        print(f"\nLearning rate is {schdler.get_last_lr()[0]}")
    print(f"\nWeights: {model.state_dict().values().mapping['0.weight'][0]}")
    print(f"\nBias:    {model.state_dict().values().mapping['0.bias'][0]}")
    print(f"\nLoss: {loss.item()}\n")

#### No Scheduler

In [33]:
train_model(scheduler=False)



Epoch :: 100 

Weights: tensor([4.0653e-03, 2.0031e+00, 3.9963e+00, 6.0017e+00, 7.9960e+00],
       device='mps:0')

Bias:    -0.0032501553650945425

Loss: 0.040374353528022766



#### StepLR

In [34]:
scheduler='StepLR'
train_model(scheduler)



Epoch :: 100 

Learning rate is 6.640349721896886e-06

Weights: tensor([2.6342e-04, 1.9987e+00, 3.9895e+00, 5.9875e+00, 7.9746e+00],
       device='mps:0')

Bias:    0.0056349434889853

Loss: 0.04118192568421364



#### CosineAnnealingLR

In [35]:
scheduler = 'CosineAnnealingLR'
train_model(scheduler)



Epoch :: 100 

Learning rate is 0.0

Weights: tensor([4.0653e-03, 2.0031e+00, 3.9963e+00, 6.0017e+00, 7.9960e+00],
       device='mps:0')

Bias:    -0.00325010041706264

Loss: 0.040374353528022766



#### ExponentialLR

In [36]:
scheduler = 'ExponentialLR'
train_model(scheduler)



Epoch :: 100 

Learning rate is 0.0014801323050834992

Weights: tensor([4.0360e-03, 2.0031e+00, 3.9963e+00, 6.0017e+00, 7.9959e+00],
       device='mps:0')

Bias:    -0.0031858838628977537

Loss: 0.04037437215447426



#### ReduceLROnPlateau

In [37]:
scheduler = 'ReduceLROnPlateau'
train_model(scheduler)



Epoch :: 100 

Learning rate is 2.5000000000000012e-08

Weights: tensor([4.0652e-03, 2.0031e+00, 3.9963e+00, 6.0017e+00, 7.9960e+00],
       device='mps:0')

Bias:    -0.0032497388310730457

Loss: 0.04037436097860336

